Data: https://www.kaggle.com/mustfkeskin/turkish-movie-sentiment-analysis-dataset/code

In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1050 Ti (UUID: GPU-ada268bc-35a4-5a51-f812-70e7584578bb)


In [2]:
import pandas as pd

df = pd.read_csv("data/turkish_movie_sentiment_dataset.csv")
df.head()

,comment,film_name,point
0,\n Jean Reno denince zate...,Sevginin Gücü,"5,0"
1,\n Ekşın falan izlemek is...,Sevginin Gücü,"5,0"
2,\n Bu yapım hakkında öyle...,Sevginin Gücü,"5,0"
3,\n finali yeter... (sting...,Sevginin Gücü,"5,0"
4,\n Jean Reno..\r\nbu adam...,Sevginin Gücü,"5,0"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83227 entries, 0 to 83226
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   comment    83227 non-null  object
 1   film_name  83227 non-null  object
 2   point      83227 non-null  object
dtypes: object(3)
memory usage: 1.9+ MB


## Preprocess

### Preprocessing guide:

* 1: Organize the dataset and comment column
* 2: Drop film_name column
* 3: Change point column's rows to integers
* 4: Remove punctuation (comment column) and lower the inputs
* 5: Remove stopwords and make sentences lower(comment column)
* 6: Stemming (comment column)

In [4]:
# Remove the unnecessary characters (space chars, \n and etc.) from the beginning and at the end
df["comment"] = df["comment"].str[23:-22]
df.head()

,comment,film_name,point
0,Jean Reno denince zaten leon filmi gelir akla ...,Sevginin Gücü,"5,0"
1,Ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü,"5,0"
2,Bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü,"5,0"
3,finali yeter... (sting - shape of my heart)\r\...,Sevginin Gücü,"5,0"
4,Jean Reno..\r\nbu adam kusursuz biri..\r\nve o...,Sevginin Gücü,"5,0"


In [5]:
# Drop "film_name" column
df.drop("film_name", axis=1, inplace=True)
df.head()

,comment,point
0,Jean Reno denince zaten leon filmi gelir akla ...,"5,0"
1,Ekşın falan izlemek istiyorsanız eğer bunu izl...,"5,0"
2,Bu yapım hakkında öyle çok şey yazabilirim ki ...,"5,0"
3,finali yeter... (sting - shape of my heart)\r\...,"5,0"
4,Jean Reno..\r\nbu adam kusursuz biri..\r\nve o...,"5,0"


### Remove punctuation

In [6]:
df['comment'] = df['comment'].str.replace(r'[^\w\s]+', '')
df.head()

C:\Users\egehan\AppData\Local\Temp/ipykernel_19128/2654021393.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['comment'] = df['comment'].str.replace(r'[^\w\s]+', '')


,comment,point
0,Jean Reno denince zaten leon filmi gelir akla ...,"5,0"
1,Ekşın falan izlemek istiyorsanız eğer bunu izl...,"5,0"
2,Bu yapım hakkında öyle çok şey yazabilirim ki ...,"5,0"
3,finali yeter sting shape of my heart\r\n\r\nb...,"5,0"
4,Jean Reno\r\nbu adam kusursuz biri\r\nve oyunc...,"5,0"


### Lower the inputs

In [7]:
df["comment"] = df["comment"].str.lower()
df.head()

,comment,point
0,jean reno denince zaten leon filmi gelir akla ...,"5,0"
1,ekşın falan izlemek istiyorsanız eğer bunu izl...,"5,0"
2,bu yapım hakkında öyle çok şey yazabilirim ki ...,"5,0"
3,finali yeter sting shape of my heart\r\n\r\nb...,"5,0"
4,jean reno\r\nbu adam kusursuz biri\r\nve oyunc...,"5,0"


### Stopwords

In [8]:
# Import the nltk library and download stopwords
import nltk

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\egehan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
# Get the stopwords
from nltk.corpus import stopwords

stop_words = stopwords.words("turkish")
stop_words[:10]

['acaba',
 'ama',
 'aslında',
 'az',
 'bazı',
 'belki',
 'biri',
 'birkaç',
 'birşey',
 'biz']

In [10]:
# Create a function to print each line of the inputs
def print_lines(column, n): 
    counter = 0
    for i in column:
        print(i)
        print("########\n")
        counter+=1
        if counter == n:
            break

print_lines(df["comment"], 3)

jean reno denince zaten leon filmi gelir akla izlemeyen kalmamıştır ama kaldıysada ee ne duruyorsun hemen izle 
########

ekşın falan izlemek istiyorsanız eğer bunu izlemeyiin dostlarım keza ilk sahne hariç ekşın filmde yerini gittikçe duygusallığa bırakır mathildanın leonun evine geldiğinde ağladığı sahnede ben de ağlamış olabilirim tamam olabilirim değil ağladım ama sen de izle ağlarsın dostooom 
etkileyici bir film sıkmıyor hiçbir salise boyunca sizi
 ben artık büyüdüm leon yaşlanıyorum 
 hayat hep böyle zor mu yoksa sadece çocukken mi 
########

bu yapım hakkında öyle çok şey yazabilirim ki kitap olur o yüzden kısa kesmem lazım bir kere ağladığım iki filmden birisidirdiğeri  yeşil yol i̇zlediğim en iyi film midir karar veremeyeceğim ama izlediğim en sanatsal sahneleri barındıran luc besson harikası olduğu kesindir 

oyunculardan sıkça bahseldilmiş o konuya girmeyeceğim ama luc besson abi sen de ne cevher varmış demekten kendimi alamıyorum yönetmenlikten çabuk çekilerek kıytırık aks

In [11]:
# Remove stopwords from each line and check the lines
stop_words = set(stop_words)
df['comment'] = df['comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

print_lines(df["comment"], 3)

jean reno denince zaten leon filmi gelir akla izlemeyen kalmamıştır kaldıysada ee duruyorsun hemen izle
########

ekşın falan izlemek istiyorsanız bunu izlemeyiin dostlarım keza ilk sahne hariç ekşın filmde yerini gittikçe duygusallığa bırakır mathildanın leonun evine geldiğinde ağladığı sahnede ben ağlamış olabilirim tamam olabilirim değil ağladım sen izle ağlarsın dostooom etkileyici bir film sıkmıyor hiçbir salise boyunca sizi ben artık büyüdüm leon yaşlanıyorum hayat böyle zor yoksa sadece çocukken mi
########

yapım hakkında öyle yazabilirim kitap olur yüzden kısa kesmem lazım bir kere ağladığım iki filmden birisidirdiğeri yeşil yol i̇zlediğim iyi film midir karar veremeyeceğim izlediğim sanatsal sahneleri barındıran luc besson harikası olduğu kesindir oyunculardan sıkça bahseldilmiş konuya girmeyeceğim luc besson abi sen cevher varmış demekten kendimi alamıyorum yönetmenlikten çabuk çekilerek kıytırık aksiyon filmlerine senaryo yazman yazık oldu gerçekten övgülerim eric serra içi

### Stemmer

In [12]:
# Import the library and get the stemmer for Turkish Language
from TurkishStemmer import TurkishStemmer

stemmer = TurkishStemmer()

In [13]:
# To use stemmer on each word, turn each line into a list
df['comment'] = df['comment'].str.split()
df.head()

,comment,point
0,"[jean, reno, denince, zaten, leon, filmi, geli...","5,0"
1,"[ekşın, falan, izlemek, istiyorsanız, bunu, iz...","5,0"
2,"[yapım, hakkında, öyle, yazabilirim, kitap, ol...","5,0"
3,"[finali, yeter, sting, shape, of, my, heart, b...","5,0"
4,"[jean, reno, adam, kusursuz, oyunculugu, müthi...","5,0"


In [14]:
# Apply stemmer
df['comment'] = df['comment'].apply(lambda x: [stemmer.stem(y) for y in x])

df.head()

,comment,point
0,"[jean, reno, deni, zaten, leon, film, gelir, a...","5,0"
1,"[ekşı, falan, izlemek, istiyor, bun, izlemey, ...","5,0"
2,"[yap, hakk, öyl, yazabilir, kitap, olur, yüz, ...","5,0"
3,"[finali, yeter, sting, shape, of, my, heart, b...","5,0"
4,"[jean, reno, ada, kusurs, oyunculug, müthiş, f...","5,0"


In [15]:
# Check the lines
print_lines(df["comment"], 3)

['jean', 'reno', 'deni', 'zaten', 'leon', 'film', 'gelir', 'akl', 'izlemeyen', 'kalm', 'kaldıysa', 'e', 'duruyor', 'hemen', 'izl']
########

['ekşı', 'falan', 'izlemek', 'istiyor', 'bun', 'izlemey', 'dost', 'keza', 'ilk', 'sahne', 'hariç', 'ekşı', 'film', 'yer', 'gittikç', 'duygusallık', 'bırakır', 'mathilda', 'leon', 'ev', 'geldik', 'ağladık', 'sahne', 'ben', 'ağla', 'olabilir', 'tama', 'olabilir', 'değil', 'ağla', 'sen', 'izl', 'ağlar', 'dostooom', 'etkileyiç', 'bir', 'film', 'sıkmıyor', 'hiçbir', 'sali', 'boyu', 'siz', 'ben', 'ar', 'büyü', 'leon', 'yaşlanıyor', 'hayat', 'böyl', 'zor', 'yoks', 'sade', 'çocukken', 'mi']
########

['yap', 'hakk', 'öyl', 'yazabilir', 'kitap', 'olur', 'yüz', 'kıs', 'kesme', 'lazım', 'bir', 'ker', 'ağladık', 'ik', 'film', 'birisidirdiğer', 'yeşil', 'yol', 'i̇zlediğim', 'iyi', 'film', 'mi', 'karar', 'veremeyecek', 'izledik', 'sanatsal', 'sahne', 'barındıran', 'luc', 'besson', 'harika', 'olduk', 'kes', 'oyuncu', 'sıkç', 'bahseldil', 'konu', 'girmeyecek', 'l

In [16]:
# Turn back each line to a string (from list)
df['comment'] = df['comment'].apply(lambda x: ' '.join(word for word in x))

df.head()

,comment,point
0,jean reno deni zaten leon film gelir akl izlem...,"5,0"
1,ekşı falan izlemek istiyor bun izlemey dost ke...,"5,0"
2,yap hakk öyl yazabilir kitap olur yüz kıs kesm...,"5,0"
3,finali yeter sting shape of my heart bazı film...,"5,0"
4,jean reno ada kusurs oyunculug müthiş film baş...,"5,0"


In [17]:
# Get train sentences from df.comment
train_sentences = df["comment"].tolist()

train_sentences[:3]

['jean reno deni zaten leon film gelir akl izlemeyen kalm kaldıysa e duruyor hemen izl',
 'ekşı falan izlemek istiyor bun izlemey dost keza ilk sahne hariç ekşı film yer gittikç duygusallık bırakır mathilda leon ev geldik ağladık sahne ben ağla olabilir tama olabilir değil ağla sen izl ağlar dostooom etkileyiç bir film sıkmıyor hiçbir sali boyu siz ben ar büyü leon yaşlanıyor hayat böyl zor yoks sade çocukken mi',
 'yap hakk öyl yazabilir kitap olur yüz kıs kesme lazım bir ker ağladık ik film birisidirdiğer yeşil yol i̇zlediğim iyi film mi karar veremeyecek izledik sanatsal sahne barındıran luc besson harika olduk kes oyuncu sıkç bahseldil konu girmeyecek luc besson abi sen cevher var demek kent alamıyor yönetmenlik çabuk çekilerek kıytırık aksiyon film senaryo yazman yazık olt gerçek övgü eric serra iç geçer nitek abartmıyor film 50 si eric serra nın hakk muhteşe melodi hayran son olarak natali portman a dek sen kadar tatl munis bir şe yahu küçük ayr bir havan şimt ayr bir havan var y

### Organize point column

In [18]:
# Replace "." with ","
df['point'] = df['point'].str.replace(',','.')
# Change data type to float
df["point"] = df["point"].astype(float)
# Rond every number and make them integers.
df["point"] = df["point"].round().astype('int')
df.head()

,comment,point
0,jean reno deni zaten leon film gelir akl izlem...,5
1,ekşı falan izlemek istiyor bun izlemey dost ke...,5
2,yap hakk öyl yazabilir kitap olur yüz kıs kesm...,5
3,finali yeter sting shape of my heart bazı film...,5
4,jean reno ada kusurs oyunculug müthiş film baş...,5


In [19]:
# Get the average and the max length of the inputs
import numpy as np

sent_lens = [len(sentence.split()) for sentence in train_sentences]
avg_sent_len = np.mean(sent_lens)
max_sent_len = np.max(sent_lens)
avg_sent_len, max_sent_len

(36.48517908851695, 3608)

In [20]:
# How long of a sentence lenght covers 95% of examples?
output_seq_len = int(np.percentile(sent_lens, 95))

output_seq_len

113

In [21]:
# How long of a sentence lenght covers 97% of examples?
x = int(np.percentile(sent_lens, 97))

x

159

In [42]:
# Setting it 160 instead of 159 since 160 is divisible to 8.
output_seq_len = 160

In [23]:
# Import AutoTokenizer
from transformers import AutoTokenizer
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")

### Test the tokizer on a sample sentence 

In [24]:
# Convert tokens to ids
sample_sentence = "Bu film berbat. Cidden en kötü film ama izlenir yine de"
sample_tokenized_sentence = tokenizer.tokenize(sample_sentence)
sample_tokenized_sentence

['bu',
 'film',
 'berbat',
 '.',
 'cidden',
 'en',
 'kotu',
 'film',
 'ama',
 'izlenir',
 'yine',
 'de']

In [25]:
# Convert ids to tokens
sample_ids = tokenizer.convert_tokens_to_ids(sample_tokenized_sentence)
sample_ids

[1964, 2767, 13686, 18, 17081, 2064, 36007, 2767, 2156, 41667, 3000, 1961]

In [26]:
sample_encoded_sentence = tokenizer.encode_plus(
    text=sample_sentence, 
    add_special_tokens=True,
    max_length=32,
    truncation=True,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors="np"
)

C:\Users\egehan\anaconda3\envs\DL\lib\site-packages\transformers\tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [27]:
# Get keys for the sample encoded sentence
sample_encoded_sentence.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [28]:
sample_input_ids = sample_encoded_sentence["input_ids"]
sample_attention_mask = sample_encoded_sentence["attention_mask"]

In [29]:
sample_input_ids[:1]

array([[    2,  1964,  2767, 13686,    18, 17081,  2064, 36007,  2767,
         2156, 41667,  3000,  1961,     3,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0]])

In [30]:
tokenizer.convert_ids_to_tokens(sample_input_ids[0])

['[CLS]',
 'bu',
 'film',
 'berbat',
 '.',
 'cidden',
 'en',
 'kotu',
 'film',
 'ama',
 'izlenir',
 'yine',
 'de',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83227 entries, 0 to 83226
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  83227 non-null  object
 1   point    83227 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 975.4+ KB


In [32]:
df.dropna(axis=0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83227 entries, 0 to 83226
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  83227 non-null  object
 1   point    83227 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 1.6+ MB


## Encode inputs

After experimenting the encoding operation on the sample sentence, now we're ready to encode our inputs (tf.comments)

In [33]:
df.comment.tolist()

['jean reno deni zaten leon film gelir akl izlemeyen kalm kaldıysa e duruyor hemen izl',
 'ekşı falan izlemek istiyor bun izlemey dost keza ilk sahne hariç ekşı film yer gittikç duygusallık bırakır mathilda leon ev geldik ağladık sahne ben ağla olabilir tama olabilir değil ağla sen izl ağlar dostooom etkileyiç bir film sıkmıyor hiçbir sali boyu siz ben ar büyü leon yaşlanıyor hayat böyl zor yoks sade çocukken mi',
 'yap hakk öyl yazabilir kitap olur yüz kıs kesme lazım bir ker ağladık ik film birisidirdiğer yeşil yol i̇zlediğim iyi film mi karar veremeyecek izledik sanatsal sahne barındıran luc besson harika olduk kes oyuncu sıkç bahseldil konu girmeyecek luc besson abi sen cevher var demek kent alamıyor yönetmenlik çabuk çekilerek kıytırık aksiyon film senaryo yazman yazık olt gerçek övgü eric serra iç geçer nitek abartmıyor film 50 si eric serra nın hakk muhteşe melodi hayran son olarak natali portman a dek sen kadar tatl munis bir şe yahu küçük ayr bir havan şimt ayr bir havan var y

In [51]:
import tensorflow as tf

input_ids = []
attention_mask = []

for txt in df.comment.values:
    encoded = tokenizer.encode_plus(
        text=txt, # the sentence to be encoded 
        add_special_tokens=True, # Add [CLS] and [SEP]
        max_length=160, # max length of a sentence
        truncation=True, # truncate if sentence length is bigger than max_length
        pad_to_max_length=True, # Add [PAD]s
        return_attention_mask=True, # Generate attention mask
        return_tensors="np" # return NumPy tensors
    )

    # Append input_ids and attention_masks to their own lists
    input_ids.append(encoded["input_ids"])
    attention_mask.append(encoded["attention_mask"])

# Concatenate
input_ids = tf.concat(input_ids, 0)
attention_mask = tf.concat(attention_mask, 0)

print("Original: ", df.comment.values[0])
print("Token IDs: ", input_ids[0])

C:\Users\egehan\anaconda3\envs\DL\lib\site-packages\transformers\tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  jean reno deni zaten leon film gelir akl izlemeyen kalm kaldıysa e duruyor hemen izl
Token IDs:  tf.Tensor(
[     2  13158  62480  78805   3671  16421   2767   3100   5855 106576
 120739  67719     47   7647   3156   2416   1019      3      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0 

In [52]:
# Convert tokens to ids to check
tokenizer.convert_ids_to_tokens(input_ids[0])

['[CLS]',
 'jean',
 'reno',
 'deni',
 'zaten',
 'leon',
 'film',
 'gelir',
 'akl',
 'izlemeyen',
 'kalm',
 'kaldıysa',
 'e',
 'duruyor',
 'hemen',
 'iz',
 '##l',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]'

In [53]:
# Check input_ids and shape of input_ids
input_ids, input_ids.shape

(<tf.Tensor: shape=(83227, 160), dtype=int32, numpy=
 array([[    2, 13158, 62480, ...,     0,     0,     0],
        [    2,  9659,  1022, ...,     0,     0,     0],
        [    2,  1997,  2533, ...,     0,     0,     0],
        ...,
        [    2, 35868,  1012, ...,     0,     0,     0],
        [    2,  6350,  3380, ...,     0,     0,     0],
        [    2,  2105,  1006, ...,     0,     0,     0]])>,
 TensorShape([83227, 160]))

In [54]:
# Check attention_mask and shape of attention_mask
attention_mask, attention_mask.shape

(<tf.Tensor: shape=(83227, 160), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])>,
 TensorShape([83227, 160]))

In [55]:
attention_mask[1], attention_mask[1].shape

(<tf.Tensor: shape=(160,), dtype=int32, numpy=
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])>,
 TensorShape([160]))

In [56]:
# Save input_ids and attention_mask
with open('movie-xids.npy', 'wb') as f:
    np.save(f, input_ids)
with open('movie-xmask.npy', 'wb') as f:
    np.save(f, attention_mask)

## One-hot-encode labels    

In [57]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
labels_one_hot = one_hot_encoder.fit_transform(df["point"].to_numpy().reshape(-1,1))
labels_one_hot

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [58]:
# Save the labels
with open("movie-labels.npy", "wb") as f:
    np.save(f, labels_one_hot)